In [2]:
import pandas as pd
path_blast = 'data/rice/2892.genome.csv'
path_blast_filtered = 'data/rice/2892.genome.filtered.csv'


In [18]:
#TEs
params = {'min_len':50,'max_len':False,'min_distance':5,'max_q':1.3,'min_q':0.7,'min_pident':97,'min_qcov':97}


In [19]:
#read blast output
df = pd.read_csv(path_blast, sep='\t', header=None)
df.columns = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs']
print('initial:',len(df.index))
initial = len(df.index)

initial: 1508


In [20]:
#filter by length
if(params['min_len']):
    df = df[df.qlen > params['min_len']]
print('Min len: ' + str(len(df.index)))
min_length = str(len(df.index))

Min len: 1508


In [21]:
if(params['max_len']):
    df = df[df.qlen < params['max_len']]
print('Max len: ' + str(len(df.index)))    
max_length = str(len(df.index))

Max len: 1508


In [22]:
#filter by query / subject length treshold
df = df[((df.length / df.qlen) >= params['min_q'])]
print('min treshold:',len(df.index))
min_treshold = str(len(df.index))

min treshold: 296


In [23]:
df = df[((df.length / df.qlen) <= params['max_q'])]
print('max treshold:',len(df.index))
max_treshold = str(len(df.index))

max treshold: 296


In [24]:
#filter by pident
df = df[(df.pident >= params['min_pident'])]
print('Min_pident: ' + str(len(df.index)))
min_pident = str(len(df.index))

Min_pident: 65


In [25]:
#filter by qcov
df = df[(df.qcovs >= params['min_qcov'])]
print('Min qcov: ' + str(len(df.index)))
min_qcov = str(len(df.index))

Min qcov: 65


In [26]:
#order sstart and send
df['new_sstart'] = df[['sstart','send']].min(axis=1)
df['new_ssend'] = df[['sstart','send']].max(axis=1)
df['sstart'] = df['new_sstart']
df['send'] = df['new_ssend']
df = df.drop('new_sstart',axis=1).drop('new_ssend',axis=1)
df = df.sort_values(by=['sseqid','sstart', 'send'])
df = df.reset_index(drop=True)
# sep by chr
dfs = {}
for seq in df.sseqid.unique():
    dfs[seq] = df[df.sseqid == seq]

In [27]:
df.head(10)

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs
0,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,80,278,4845501,4845699,5,0,97.487,4.050000e-91,199,278,43270923,100
1,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,1,278,6554388,6554665,3,0,98.921,1.020000e-136,278,278,43270923,100
2,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,1,278,8009459,8009736,4,0,98.561,1.240000e-135,278,278,43270923,100
3,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,1,278,8780736,8781013,4,0,98.561,1.240000e-135,278,278,43270923,100
4,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,1,275,11569000,11569274,3,0,98.909,4.320000e-135,275,278,43270923,100
5,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,1,278,15197067,15197344,2,0,99.281,2.390000e-138,278,278,43270923,100
6,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,1,278,19494346,19494623,3,0,98.921,1.020000e-136,278,278,43270923,100
7,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,80,278,20565509,20565707,5,0,97.487,4.050000e-91,199,278,43270923,100
8,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,1,278,31194947,31195224,0,0,100.000,4.610000e-141,278,278,43270923,100
9,MITE_T_2892|chr03|12410305|12410615|TA|38|F242...,chr01,1,278,33042295,33042572,2,0,99.281,2.390000e-138,278,278,43270923,100


In [28]:
# filter overlapped 
rows = []
discard = []
total = len(df.index)
count = 0
curr = 0
for index, row in df.iterrows():
    count += 1
    curr_new = int(count * 100 * 1.0 / (total * 1.0))
    if curr_new != curr:
        curr = curr_new
        if curr_new % 1 == 0:
            print(curr_new)
    if index in discard:
        continue
    for k2, v2 in df.loc[index:,].iterrows():
        if abs(v2.sstart - row.sstart) > params['min_distance']:
            break
        if abs(v2.sstart - row.sstart) <= params['min_distance'] and abs(v2.send - row.send) <= params['min_distance']:
            discard.append(k2)
    rows.append(row)


1
3
4
6
7
9
10
12
13
15
16
18
20
21
23
24
26
27
29
30
32
33
35
36
38
40
41
43
44
46
47
49
50
52
53
55
56
58
60
61
63
64
66
67
69
70
72
73
75
76
78
80
81
83
84
86
87
89
90
92
93
95
96
98
100


In [29]:
df = pd.DataFrame(rows)
print('Non overlapped: ' + str(len(df.index)))
non_overlapped = str(len(df.index))


Non overlapped: 65


In [30]:
df.to_csv(path_blast_filtered, index=None, sep='\t')
path_blast_filtered

'data/rice/2892.genome.filtered.csv'

In [31]:
print('Initial: ' + str(initial))
print('Min len: ' + str(min_length))
print('Max len: ' + str(max_length))
print('Min treshold: ' + str(min_treshold))
print('Max treshold: ' + str(max_treshold))
print('Min pident: ' + str(min_pident))
print('Min qcov: ' + str(min_qcov))
print('Non overlapped: ' + str(non_overlapped))
print('Saved: ' + path_blast_filtered)

Initial: 1508
Min len: 1508
Max len: 1508
Min treshold: 296
Max treshold: 296
Min pident: 65
Min qcov: 65
Non overlapped: 65
Saved: data/rice/2892.genome.filtered.csv
